In [14]:
import pydicom
import os

dir = '/Users/oovamoyo/Downloads/МРТ Липатова ТА - БП+ЭТ/2023'
fname = os.path.join(dir, 'DICOMDIR')
dcom = pydicom.dcmread(fname)

In [15]:
dcom.DirectoryRecordSequence

<Sequence, length 416>

In [67]:
print(len(dcom.DirectoryRecordSequence))
# 1. create a new set
record_types = set()

# 2. populate with DirectoryRecordType
for rec in dcom.DirectoryRecordSequence:
    record_types.add(rec.DirectoryRecordType)

# Print the unique DirectoryRecordTypes
print(record_types)


for record in dcom.DirectoryRecordSequence:
    if record.DirectoryRecordType == 'IMAGE':
        print(record)
        break
        # path = os.path.join(*record.ReferencedFileID)
        # print(path)

416
{'PATIENT', 'IMAGE', 'STUDY', 'SERIES'}
(0004,1400) Offset of the Next Directory Record UL: 6622
(0004,1410) Record In-use Flag                  US: 65535
(0004,1420) Offset of Referenced Lower-Level Di UL: 0
(0004,1430) Directory Record Type               CS: 'IMAGE'
(0004,1500) Referenced File ID                  CS: ['DICOM', '23022710', '08050000', '71646759']
(0004,1510) Referenced SOP Class UID in File    UI: MR Image Storage
(0004,1511) Referenced SOP Instance UID in File UI: 1.3.12.2.1107.5.2.6.23587.30000023022705114062500001131
(0004,1512) Referenced Transfer Syntax UID in F UI: Explicit VR Little Endian
(0008,0005) Specific Character Set              CS: 'ISO_IR 100'
(0008,0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'M', 'ND']
(0008,0016) SOP Class UID                       UI: MR Image Storage
(0008,0018) SOP Instance UID                    UI: 1.3.12.2.1107.5.2.6.23587.30000023022705114062500001131
(0008,0023) Content Date                     

In [ ]:
import yadisk

YA_TOKEN=os.getenv('YA_TOKEN')
client = yadisk.Client(token=YA_TOKEN)

print('Token is valid?', client.check_token())

Token is valid? True


In [31]:
client.mkdir('Загрузки/MRT_PNGs')

<SyncResourceLinkObject{
    'href': 'https://cloud-api.yandex.net/v1/disk/resources?path=disk%3A%2F%D0%97%D0%B0%D0%B3%D1%80%D1%83%D0%B7%D0%BA%D0%B8%2FMRT_PNGs',
    'method': 'GET',
    'templated': False,
    'path': 'disk:/Загрузки/MRT_PNGs'})>

In [ ]:
client.upload(os.path.join(os.getcwd(), 'artifacts/MPT_AAA_PNGs/2024'), 'Загрузки/MRT_PNGs/2024', overwrite=True)

IsADirectoryError: [Errno 21] Is a directory: '/Users/oovamoyo/PycharmProjects/brain_pd_et/artifacts/MPT_AAA_PNGs/2024'

In [37]:
from PIL import Image
import numpy as np

im = Image.open('artifacts/MPT_AAA_PNGs/2024/DICOM/PA000000/ST000000/SE000004/IM000001.png')
arr = np.asarray(im)
arr.shape

(128, 128)

In [ ]:
from collections import deque
from typing import Optional

def check_all_folders_for_dicomdir(cl: yadisk.Client, public_url: str, path: str = ''):
    patients = cl.public_listdir(public_url, path='', sort='name')
    for patient in patients:
        hasdicomdir = search_and_destroy(cl, public_url, patient.path)
        if not hasdicomdir:
            print(f'No DICOMDIR found for patient folder: {patient.path}')

def search_and_destroy(cl: yadisk.Client, public_url: str, patient: str) -> list:
    """ Returns a list of paths to all DICOMDIR files found. """
    dicomdirs = []
    queue = deque([patient])
    while queue:
        current_path = queue.popleft()
        for item in cl.public_listdir(public_url, path=current_path, sort='name'):
            if item.type == "dir":
                # Add subdirectories to the queue
                queue.append(item.path)
            elif item.type == "file" and item.name.upper() == 'DICOMDIR':
                dicomdirs.append(item.path)
    return dicomdirs



In [ ]:
check_all_folders_for_dicomdir(client, os.getenv('SOURCE_PUBLIC_URL'))

No DICOMDIR found for patient folder: /МРТ Архипова ОВ - ранняя БП
No DICOMDIR found for patient folder: /МРТ Вагапов ФА-СК
No DICOMDIR found for patient folder: /МРТ Гаврилова НМ - БП+ЭТ
No DICOMDIR found for patient folder: /МРТ Гатауллин ЗГ-БП (МСАпод вопросом)
No DICOMDIR found for patient folder: /МРТ Гафиатуллина МФ - ЭТ + БП+ХИГМ
No DICOMDIR found for patient folder: /МРТ Ильина НМ - БП
No DICOMDIR found for patient folder: /МРТ Королев СП - БП
No DICOMDIR found for patient folder: /МРТ Садреева ГР - БП
No DICOMDIR found for patient folder: /МРТ Филиппов НВ - БП
No DICOMDIR found for patient folder: /МРТ Шкильменская ЗМ - насл. БП


WrongResourceTypeError: 'https://disk.yandex.ru/d/INOsQEAaSA8QcA' is a file